<a href="https://colab.research.google.com/github/tatkoch/MOSiR_Lancut/blob/main/MOSiR_wykresy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import plotly.graph_objects as go

# Wczytaj dane z GitHub
url = "https://raw.githubusercontent.com/tatkoch/MOSiR_Lancut/main/frekwencja.csv"
df = pd.read_csv(url, skiprows=1, names=["timestamp", "aktualna", "maksymalna", "temperatura", "warunki", "prognoza"])

# Parsowanie i czyszczenie danych
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["aktualna"] = pd.to_numeric(df["aktualna"], errors="coerce")
df.dropna(subset=["timestamp", "aktualna"], inplace=True)
df = df[df["aktualna"] > 0]

# Kolumny pomocnicze
df["data"] = df["timestamp"].dt.date
df["godzina"] = df["timestamp"].dt.hour
df["dzien_en"] = df["timestamp"].dt.day_name()

# Tłumaczenie dni tygodnia na polski
dni_polskie = {
    "Monday": "Poniedziałek",
    "Tuesday": "Wtorek",
    "Wednesday": "Środa",
    "Thursday": "Czwartek",
    "Friday": "Piątek",
    "Saturday": "Sobota",
    "Sunday": "Niedziela"
}
df["dzien_pl"] = df["dzien_en"].map(dni_polskie)

dni_kolejnosc = list(dni_polskie.values())
fig = go.Figure()
linie_dzien = []

# Dodaj ślady dla każdego dnia tygodnia
for dzien in dni_kolejnosc:
    df_dzien = df[df["dzien_pl"] == dzien]
    for data in sorted(df_dzien["data"].unique()):
        df_data = df_dzien[df_dzien["data"] == data]
        fig.add_trace(go.Scatter(
            x=df_data["godzina"],
            y=df_data["aktualna"],
            mode="lines+markers",
            name=f"{data} ({dzien})",
            visible=False
        ))
        linie_dzien.append((dzien, data))

# Linie poziome: 80 i 70
godziny = list(range(6, 23))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[80]*len(godziny),
    mode="lines",
    name="Maksymalna (80)",
    line=dict(color="red", dash="dash"),
    visible=True
))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[70]*len(godziny),
    mode="lines",
    name="Ryzykowna (70)",
    line=dict(color="orange", dash="dot"),
    visible=True
))

# Dropdown: dni tygodnia + wszystkie
buttons = []
offset = 0
for dzien in dni_kolejnosc:
    count = sum(1 for d, _ in linie_dzien if d == dzien)
    visible = [False] * len(fig.data)
    for i in range(offset, offset + count):
        visible[i] = True
    visible[-2] = True
    visible[-1] = True
    buttons.append(dict(label=dzien, method="update", args=[{"visible": visible}, {"title": f"Frekwencja — {dzien}"}]))
    offset += count

visible_all = [True] * (len(fig.data) - 2) + [True, True]
buttons.append(dict(label="Wszystkie dni", method="update", args=[{"visible": visible_all}, {"title": "Frekwencja — Wszystkie dni"}]))

fig.update_layout(
    title="Frekwencja — Wszystkie dni",
    xaxis=dict(
        tickmode="array",
        tickvals=list(range(6, 23)),
        title="Godzina",
        range=[6, 22]
    ),
    yaxis_title="Liczba osób",
    updatemenus=[dict(active=len(buttons)-1, buttons=buttons, x=0.1, y=1.15)],
    height=600
)

fig.show()


In [4]:
import pandas as pd
import plotly.graph_objects as go

# Wczytaj dane z GitHub
url = "https://raw.githubusercontent.com/tatkoch/MOSiR_Lancut/main/frekwencja.csv"
df = pd.read_csv(url, skiprows=1, names=["timestamp", "aktualna", "maksymalna", "temperatura", "warunki", "prognoza"])

# Parsowanie i czyszczenie danych
df["timestamp"] = pd.to_datetime(df["timestamp"], format="%Y-%m-%d %H:%M:%S", errors="coerce")
df["aktualna"] = pd.to_numeric(df["aktualna"], errors="coerce")
df.dropna(subset=["timestamp", "aktualna"], inplace=True)
df = df[df["aktualna"] > 0]

# Kolumny pomocnicze
df["godzina"] = df["timestamp"].dt.hour
df["dzien_en"] = df["timestamp"].dt.day_name()

dni_polskie = {
    "Monday": "Poniedziałek",
    "Tuesday": "Wtorek",
    "Wednesday": "Środa",
    "Thursday": "Czwartek",
    "Friday": "Piątek",
    "Saturday": "Sobota",
    "Sunday": "Niedziela"
}
df["dzien_pl"] = df["dzien_en"].map(dni_polskie)
dni_kolejnosc = list(dni_polskie.values())

fig = go.Figure()

# Dodaj średnie dla każdego dnia
for i, dzien in enumerate(dni_kolejnosc):
    df_dzien = df[df["dzien_pl"] == dzien]
    srednia = df_dzien.groupby("godzina")["aktualna"].mean().reindex(range(6, 23))
    fig.add_trace(go.Scatter(
        x=srednia.index,
        y=srednia.values,
        mode="lines+markers",
        name=dzien,
        visible=(i == 0)
    ))

# Linie poziome: 80 i 70
godziny = list(range(6, 23))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[80]*len(godziny),
    mode="lines",
    name="Maksymalna (80)",
    line=dict(color="red", dash="dash"),
    visible=True
))
fig.add_trace(go.Scatter(
    x=godziny,
    y=[70]*len(godziny),
    mode="lines",
    name="Ryzykowna (70)",
    line=dict(color="orange", dash="dot"),
    visible=True
))

# Dropdown
buttons = []
for i, dzien in enumerate(dni_kolejnosc):
    visible = [False] * len(fig.data)
    visible[i] = True
    visible[-2] = True
    visible[-1] = True
    buttons.append(dict(label=dzien, method="update", args=[{"visible": visible}, {"title": f"Średnia frekwencja — {dzien}"}]))

visible_all = [True] * len(dni_kolejnosc) + [True, True]
buttons.append(dict(label="Wszystkie dni", method="update", args=[{"visible": visible_all}, {"title": "Średnia frekwencja — Wszystkie dni"}]))

fig.update_layout(
    title="Średnia frekwencja — Wszystkie dni",
    xaxis=dict(
        tickmode="array",
        tickvals=list(range(6, 23)),
        title="Godzina",
        range=[6, 22]
    ),
    yaxis_title="Liczba osób",
    updatemenus=[dict(active=len(buttons)-1, buttons=buttons, x=0.1, y=1.15)],
    height=600
)

fig.show()
